In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/application_train', './input/application_test', './input/bureau',
                './input/bureau_balance']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

In [1]:
# import pandas as pd 
exec(os.environ['IREWR_IMPORTS'])
import numpy as np
# FIRST-AUTHOR: remove plotting
# import matplotlib
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
# FIRST-AUTHOR: remove ML code
# from sklearn.metrics import roc_auc_score
from IPython.display import display
# FIRST-AUTHOR: remove plotting
# %matplotlib inline 
data_path="./input"

In [2]:
data = pd.concat([pd.read_csv(os.path.join(data_path, "application_train.scaled.csv"), index_col="SK_ID_CURR"), pd.read_csv(os.path.join(data_path, "application_test.scaled.csv"), index_col="SK_ID_CURR")], axis=0)
data.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
data.TARGET.fillna(-1).value_counts()

 0.0    282686
-1.0     48744
 1.0     24825
Name: TARGET, dtype: int64

In [4]:
for c in data.select_dtypes(include="object"):
    data[c] = data[c].astype("category")

In [5]:
data.dtypes.value_counts()

float64     66
int64       39
category     2
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
dtype: int64

In [6]:
bureau = pd.read_csv(os.path.join(data_path, "bureau.scaled.csv"))
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [7]:
bureau_balance = pd.read_csv(os.path.join(data_path, "bureau_balance.scaled.csv")).join(bureau.set_index("SK_ID_BUREAU").SK_ID_CURR, on="SK_ID_BUREAU")

In [8]:
app = bureau[bureau.SK_ID_CURR == 100002]
display(app)
display(bureau_balance[bureau_balance.SK_ID_BUREAU.isin(app.SK_ID_BUREAU.unique())])

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
675684,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0
675685,100002,6158905,Closed,currency 1,-476,0,NaN,-48.0,NaN,0,0.000,0.0,NaN,0.0,Credit card,-47,NaN
675686,100002,6158906,Closed,currency 1,-1437,0,-1072.0,-1185.0,0.000,0,135000.000,0.0,0.000,0.0,Consumer credit,-1185,0.0
675687,100002,6158907,Closed,currency 1,-1121,0,-911.0,-911.0,3321.000,0,19071.000,NaN,NaN,0.0,Consumer credit,-906,0.0
675688,100002,6158908,Closed,currency 1,-645,0,85.0,-36.0,5043.645,0,120735.000,0.0,0.000,0.0,Consumer credit,-34,0.0
675689,100002,6158909,Active,currency 1,-103,0,NaN,NaN,40.500,0,31988.565,0.0,31988.565,0.0,Credit card,-24,0.0
1337779,100002,6158903,Active,currency 1,-1042,0,780.0,NaN,NaN,0,450000.000,245781.0,0.000,0.0,Consumer credit,-7,0.0
1486113,100002,6113835,Closed,currency 1,-1043,0,62.0,-967.0,0.000,0,67500.000,NaN,NaN,0.0,Credit card,-758,0.0


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,SK_ID_CURR
13472782,6158904,-15,C,100002.0
13472783,6158904,-16,C,100002.0
13472784,6158904,-17,0,100002.0
13472785,6158904,-18,0,100002.0
13472786,6158904,-19,0,100002.0
...,...,...,...,...
22638288,6113835,-30,0,100002.0
22638289,6113835,-31,0,100002.0
22638290,6113835,-32,0,100002.0
22638291,6113835,-33,X,100002.0


In [9]:
bureau_balance_dumies = pd.get_dummies(bureau_balance.STATUS)
bureau_balance_dumies["SK_ID_CURR"] = bureau_balance.SK_ID_CURR
bureau_balance_dumies = bureau_balance_dumies.groupby("SK_ID_CURR").sum()

In [10]:
irregular = ["bureau_balance_status_" + c for c in bureau_balance_dumies if c not in ["0", "C", "X"]]
very_irregular = ["bureau_balance_status_" + c for c in bureau_balance_dumies if c not in ["0", "1", "2", "C", "X"]]

In [11]:
bureau_balance_dumies.columns = ["bureau_balance_status_" + c for c in bureau_balance_dumies]

In [12]:
bureau_balance_dumies["total_balances"] = bureau_balance_dumies.sum(axis=1)
bureau_balance_dumies["irregular_bureau_balance"] = bureau_balance_dumies[irregular].sum(axis=1)
bureau_balance_dumies["irregular_bureau_balance_ratio"] = bureau_balance_dumies["irregular_bureau_balance"] / bureau_balance_dumies["total_balances"]
bureau_balance_dumies["very_irregular_bureau_balance"] = bureau_balance_dumies[very_irregular].sum(axis=1)
bureau_balance_dumies["very_irregular_bureau_balance_ratio"] = bureau_balance_dumies["very_irregular_bureau_balance"] / bureau_balance_dumies["total_balances"]
bureau_balance_dumies["super_irregular_bureau_balance_ratio"] = bureau_balance_dumies["bureau_balance_status_5"] / bureau_balance_dumies["total_balances"]
bureau_balance_dumies.head(100)

,bureau_balance_status_0,bureau_balance_status_1,bureau_balance_status_2,bureau_balance_status_3,bureau_balance_status_4,bureau_balance_status_5,bureau_balance_status_C,bureau_balance_status_X,total_balances,irregular_bureau_balance,irregular_bureau_balance_ratio,very_irregular_bureau_balance,very_irregular_bureau_balance_ratio,super_irregular_bureau_balance_ratio
SK_ID_CURR,,,,,,,,,,,,,,
100001.0,31,1,0,0,0,0,110,30,172,1,0.005814,0,0.0,0.0
100002.0,45,27,0,0,0,0,23,15,110,27,0.245455,0,0.0,0.0
100005.0,14,0,0,0,0,0,5,2,21,0,0.000000,0,0.0,0.0
100010.0,20,0,0,0,0,0,52,0,72,0,0.000000,0,0.0,0.0
100013.0,79,7,0,0,0,0,103,41,230,7,0.030435,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100243.0,33,0,0,0,0,0,12,27,72,0,0.000000,0,0.0,0.0
100245.0,145,2,0,0,0,0,247,153,547,2,0.003656,0,0.0,0.0
100247.0,290,10,0,0,0,0,57,18,375,10,0.026667,0,0.0,0.0


In [13]:
extended_data = [bureau_balance_dumies]
del bureau_balance_dumies, bureau_balance

In [14]:
for c in bureau.select_dtypes(["object"]):
    display(bureau[c].value_counts(dropna=False).to_frame())

,CREDIT_ACTIVE
Closed,1079273
Active,630607
Sold,6527
Bad debt,21


,CREDIT_CURRENCY
currency 1,1715020
currency 2,1224
currency 3,174
currency 4,10


,CREDIT_TYPE
Consumer credit,1251615
Credit card,402195
Car loan,27690
Mortgage,18391
Microloan,12413
Loan for business development,1975
Another type of loan,1017
Unknown type of loan,555
Loan for working capital replenishment,469
Cash loan (non-earmarked),56


In [15]:
bureau.groupby("SK_ID_CURR").CREDIT_CURRENCY.unique().apply(len).value_counts()

1    304739
2      1045
3        27
Name: CREDIT_CURRENCY, dtype: int64

In [16]:
credits_status = pd.get_dummies(bureau.CREDIT_ACTIVE)
credits_status["SK_ID_CURR"] = bureau.SK_ID_CURR
# FIRST-AUTHOR: make notebook run
# credits_status = credits_status.groupby("SK_ID_CURR").sum(axis=0)
credits_status = credits_status.groupby("SK_ID_CURR").sum()

credits_status.columns = ["credits_status_" + "_".join(c.lower().split()) for c in credits_status]
total_credits = credits_status.sum(axis=1)

credits_status["sold_or_bad_credits"] = credits_status[["credits_status_bad_debt", "credits_status_sold"]].sum(axis=1)

ratios = pd.concat(((credits_status[c] / total_credits).rename(c + "_ratio") for c in credits_status), axis=1)

credits_status["total_credits"] = total_credits
credits_status = credits_status.join(ratios)
credits_status

,credits_status_active,credits_status_bad_debt,credits_status_closed,credits_status_sold,sold_or_bad_credits,total_credits,credits_status_active_ratio,credits_status_bad_debt_ratio,credits_status_closed_ratio,credits_status_sold_ratio,sold_or_bad_credits_ratio
SK_ID_CURR,,,,,,,,,,,
100001,3,0,4,0,0,7,0.428571,0.0,0.571429,0.0,0.0
100002,2,0,6,0,0,8,0.250000,0.0,0.750000,0.0,0.0
100003,1,0,3,0,0,4,0.250000,0.0,0.750000,0.0,0.0
100004,0,0,2,0,0,2,0.000000,0.0,1.000000,0.0,0.0
100005,2,0,1,0,0,3,0.666667,0.0,0.333333,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
456249,2,0,11,0,0,13,0.153846,0.0,0.846154,0.0,0.0
456250,2,0,1,0,0,3,0.666667,0.0,0.333333,0.0,0.0
456253,2,0,2,0,0,4,0.500000,0.0,0.500000,0.0,0.0


In [17]:
extended_data.append(credits_status)
del credits_status

In [18]:
credits_types = pd.get_dummies(bureau.CREDIT_TYPE)
credits_types["SK_ID_CURR"] = bureau.SK_ID_CURR
# FIRST-AUTHOR: make notebook run
# credits_types = credits_types.groupby("SK_ID_CURR").sum(axis=0)
credits_types = credits_types.groupby("SK_ID_CURR").sum()
credits_types.columns = ["credits_types_" + "_".join(c.lower().split()) for c in credits_types]
credits_types

,credits_types_another_type_of_loan,credits_types_car_loan,credits_types_cash_loan_(non-earmarked),credits_types_consumer_credit,credits_types_credit_card,credits_types_interbank_credit,credits_types_loan_for_business_development,credits_types_loan_for_purchase_of_shares_(margin_lending),credits_types_loan_for_the_purchase_of_equipment,credits_types_loan_for_working_capital_replenishment,credits_types_microloan,credits_types_mobile_operator_loan,credits_types_mortgage,credits_types_real_estate_loan,credits_types_unknown_type_of_loan
SK_ID_CURR,,,,,,,,,,,,,,,
100001,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0
100002,0,0,0,4,4,0,0,0,0,0,0,0,0,0,0
100003,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0
100004,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0
100005,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,1,0,0,9,3,0,0,0,0,0,0,0,0,0,0
456250,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0
456253,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0


In [19]:
extended_data.append(credits_types)
del credits_types

In [20]:
bureau.DAYS_ENDDATE_FACT.fillna(bureau.DAYS_CREDIT_ENDDATE, inplace=True)
bureau.DAYS_CREDIT_ENDDATE.fillna(bureau.DAYS_ENDDATE_FACT, inplace=True)
bureau.DAYS_ENDDATE_FACT.fillna(0, inplace=True)
bureau.DAYS_CREDIT_ENDDATE.fillna(0, inplace=True)
bureau["days_end_credit_advancement"] = bureau.DAYS_CREDIT_ENDDATE - bureau.DAYS_ENDDATE_FACT
max_cols = ["CREDIT_DAY_OVERDUE", "AMT_CREDIT_MAX_OVERDUE", "CNT_CREDIT_PROLONG", "days_end_credit_advancement"]
bureau_agg_data = bureau[max_cols + ["SK_ID_CURR"]].fillna(0).groupby("SK_ID_CURR").max()
bureau_agg_data.columns = ["max_" + "_".join(c.lower().split()) for c in bureau_agg_data]
display(bureau_agg_data)


,max_credit_day_overdue,max_amt_credit_max_overdue,max_cnt_credit_prolong,max_days_end_credit_advancement
SK_ID_CURR,,,,
100001,0,0.000,0,698.0
100002,0,5043.645,0,1029.0
100003,0,0.000,0,201.0
100004,0,0.000,0,88.0
100005,0,0.000,0,0.0
...,...,...,...,...
456249,0,18945.000,0,59.0
456250,0,0.000,0,488.0
456253,0,0.000,0,605.0


In [21]:
extended_data.append(bureau_agg_data)
del bureau_agg_data

In [22]:
sum_cols = ["CREDIT_DAY_OVERDUE", "AMT_CREDIT_MAX_OVERDUE", "CNT_CREDIT_PROLONG", "days_end_credit_advancement", "AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT", "AMT_CREDIT_SUM_LIMIT", "AMT_CREDIT_SUM_OVERDUE"]
# FIRST-AUTHOR: make notebook run
# bureau_agg_data = bureau[sum_cols + ["SK_ID_CURR"]].fillna(0).groupby("SK_ID_CURR").sum(axis=0)
bureau_agg_data = bureau[sum_cols + ["SK_ID_CURR"]].fillna(0).groupby("SK_ID_CURR").sum()
bureau_agg_data["total_outstanding_debt_ratio"] = bureau_agg_data.AMT_CREDIT_SUM_DEBT / bureau_agg_data.AMT_CREDIT_SUM
bureau_agg_data.columns = ["sum_" + "_".join(c.lower().split()) for c in bureau_agg_data]
display(bureau_agg_data)


,sum_credit_day_overdue,sum_amt_credit_max_overdue,sum_cnt_credit_prolong,sum_days_end_credit_advancement,sum_amt_credit_sum,sum_amt_credit_sum_debt,sum_amt_credit_sum_limit,sum_amt_credit_sum_overdue,sum_total_outstanding_debt_ratio
SK_ID_CURR,,,,,,,,,
100001,0,0.000,0,788.0,1453365.000,596686.500,0.000,0.0,0.410555
100002,0,8405.145,0,1263.0,865055.565,245781.000,31988.565,0.0,0.284122
100003,0,0.000,0,-102.0,1017400.500,0.000,810000.000,0.0,0.000000
100004,0,0.000,0,88.0,189037.800,0.000,0.000,0.0,0.000000
100005,0,0.000,0,-5.0,657126.000,568408.500,0.000,0.0,0.864992
...,...,...,...,...,...,...,...,...,...
456249,0,30735.000,0,-65.0,3693858.660,163071.000,0.000,0.0,0.044147
456250,0,0.000,0,488.0,3086459.550,2232040.095,58268.385,0.0,0.723172
456253,0,0.000,0,1210.0,3960000.000,1795833.000,0.000,0.0,0.453493


In [23]:
extended_data.append(bureau_agg_data)
del bureau_agg_data

In [24]:
sum_cols = ["AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT"]
# FIRST-AUTHOR: make notebook run
# active_bureau =  bureau[bureau.CREDIT_ACTIVE == "Active"][sum_cols + ["SK_ID_CURR"]].groupby("SK_ID_CURR").sum(axis=0)
active_bureau =  bureau[bureau.CREDIT_ACTIVE == "Active"][sum_cols + ["SK_ID_CURR"]].groupby("SK_ID_CURR").sum()
active_bureau.columns = ["total_active_debts", "outstanding_active_debts"]
display(active_bureau)
extended_data.append(active_bureau)
del active_bureau

,total_active_debts,outstanding_active_debts
SK_ID_CURR,,
100001,884025.000,596686.500
100002,481988.565,245781.000
100003,810000.000,0.000
100005,598626.000,568408.500
100008,267606.000,240057.000
...,...,...
456247,4113000.000,2193390.000
456249,405000.000,163071.000
456250,2603110.050,2232040.095


In [25]:
data = data.join(pd.concat(extended_data, axis=1))
display(data)
del extended_data

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,sum_amt_credit_max_overdue,sum_cnt_credit_prolong,sum_days_end_credit_advancement,sum_amt_credit_sum,sum_amt_credit_sum_debt,sum_amt_credit_sum_limit,sum_amt_credit_sum_overdue,sum_total_outstanding_debt_ratio,total_active_debts,outstanding_active_debts
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002.0,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,8405.145,0.0,1263.0,865055.565,245781.000,31988.565,0.0,0.284122,481988.565,245781.000
100003.0,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0.000,0.0,-102.0,1017400.500,0.000,810000.000,0.0,0.000000,810000.000,0.000
100004.0,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0.000,0.0,88.0,189037.800,0.000,0.000,0.0,0.000000,NaN,NaN
100006.0,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100007.0,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0.000,0.0,0.0,146250.000,0.000,0.000,0.0,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456221.0,NaN,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,22855.500,0.0,120.0,1118799.000,556470.000,0.000,0.0,0.497382,837931.500,556470.000
456222.0,NaN,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456223.0,NaN,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0.000,0.0,122.0,2156309.460,21343.500,29202.750,0.0,0.009898,50547.960,21343.500


In [26]:
data["credit_goodsprice_ratio"] = data.AMT_CREDIT / data.AMT_GOODS_PRICE
data["annuity_income_ratio"] = data.AMT_ANNUITY / data.AMT_INCOME_TOTAL
data["annuity_credit_ratio"] = data.AMT_ANNUITY / data.AMT_CREDIT
data["total_new_credit_increase"] = data.AMT_CREDIT + data.outstanding_active_debts
data["credit_increase"] = data.AMT_CREDIT / data.total_new_credit_increase
data["total_new_credit_income_ratio"] = data.total_new_credit_increase / data.AMT_INCOME_TOTAL

In [27]:
data.head(100)

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,sum_amt_credit_sum_overdue,sum_total_outstanding_debt_ratio,total_active_debts,outstanding_active_debts,credit_goodsprice_ratio,annuity_income_ratio,annuity_credit_ratio,total_new_credit_increase,credit_increase,total_new_credit_income_ratio
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002.0,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0.0,0.284122,481988.565,245781.00,1.158397,0.121978,0.060749,652378.50,0.623254,3.221622
100003.0,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0.0,0.000000,810000.000,0.00,1.145199,0.132217,0.027598,1293502.50,1.000000,4.790750
100004.0,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0.0,0.000000,NaN,NaN,1.000000,0.100000,0.050000,NaN,NaN,NaN
100006.0,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,NaN,NaN,NaN,NaN,1.052803,0.219900,0.094941,NaN,NaN,NaN
100007.0,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0.0,0.000000,NaN,NaN,1.000000,0.179963,0.042623,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100113.0,0.0,Cash loans,M,Y,N,0,76500.0,135000.0,6696.0,135000.0,...,0.0,0.657203,567000.000,401368.50,1.000000,0.087529,0.049600,536368.50,0.251693,7.011353
100114.0,0.0,Cash loans,F,N,Y,0,135000.0,284400.0,22599.0,225000.0,...,0.0,0.195205,142445.880,45634.50,1.264000,0.167400,0.079462,330034.50,0.861728,2.444700
100115.0,0.0,Cash loans,F,N,N,0,90000.0,315000.0,14683.5,315000.0,...,0.0,0.000000,NaN,NaN,1.000000,0.163150,0.046614,NaN,NaN,NaN


In [28]:
# FIRST-AUTHOR: remove ML code
# from lightgbm import LGBMClassifier
def train_model(train, target, nl, X_test, folds=5):
    test_probs = []
    for i in range(folds):
        X_valid = train.sample(frac=1/folds)
        y_valid = X_valid[target]
        X_valid = X_valid.drop(target, axis=1)
        
        X_train = train.drop(X_valid.index)
        y_train = X_train[target]
        X_train = X_train.drop(target, axis=1)

# FIRST-AUTHOR: remove ML code
#         learner = LGBMClassifier(n_estimators=10000, num_leaves=nl)
#         learner.fit(X_train, y_train,  early_stopping_rounds=10, eval_metric="auc", verbose=50,
#                     eval_set=[(X_train, y_train),
#                               (X_valid, y_valid)])
#         probs = pd.Series(learner.predict_proba(X_test)[:, -1], index=X_test.index, name="fold_" + str(i))
        probs = pd.Series(y_train, index=X_test.index, name="fold_" + str(i))
        test_probs.append(probs)
    return pd.concat(test_probs, axis=1).mean(axis=1)

In [29]:
test = data[data.TARGET.notnull()].sample(frac=0.1)
train = data[data.TARGET.notnull()].drop(test.index)
X_test = test.drop("TARGET", axis=1)
y_test = test.TARGET

nls = [2 ** i for i in [4, 5, 6]]
res = pd.Series([np.nan] * len(nls), index=nls, name="ROC_AUC")
for nl in nls:
    print("*"*10, nl, "*"*10)
    probs = train_model(train, "TARGET", nl, X_test, 10)
# FIRST-AUTHOR: remove ML code
#     res.loc[nl] = roc_auc_score(y_test, probs)
    res.loc[nl] = 0.5
    print("ROC_AUC para {nl} hojas: {res:.4f}".format(nl=nl, res=res.loc[nl]))

********** 16 **********
ROC_AUC para 16 hojas: 0.5000
********** 32 **********
ROC_AUC para 32 hojas: 0.5000
********** 64 **********
ROC_AUC para 64 hojas: 0.5000


In [30]:
res.to_frame()

,ROC_AUC
16,0.5
32,0.5
64,0.5


In [31]:
train = data[data.TARGET.notnull()]
X_test = data.drop(train.index).drop("TARGET", axis=1)
nl = res.idxmax()
train_model(train, "TARGET", nl, X_test).rename("TARGET").to_csv("submission_{nl}.csv".format(nl=nl), header=True)